# card_stack

> Card stack UI operations — navigation, viewport, and response builders

In [ ]:
#| default_exp routes.card_stack

In [ ]:
#| export
from typing import List, Dict, Any, Tuple, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.models import CardStackState, CardStackUrls
from cjm_fasthtml_card_stack.core.constants import DEFAULT_CARD_WIDTH
from cjm_fasthtml_card_stack.routes.handlers import (
    build_slots_response, build_nav_response,
    card_stack_navigate, card_stack_update_viewport, card_stack_save_width,
)

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.components.card_stack_config import (
    REVIEW_CS_CONFIG, REVIEW_CS_IDS,
)
from cjm_transcript_review.components.review_card import (
    create_review_card_renderer, AssembledSegment
)
from cjm_transcript_review.routes.core import (
    WorkflowStateStore, _load_review_context, _get_review_state,
    _build_card_stack_state, _update_review_state, _get_assembled_segments,
)

## Response Builders

Shared helpers that assemble OOB response tuples for card stack operations.

In [ ]:
#| export
def _build_slots_oob(
    assembled:List[AssembledSegment],  # Assembled segments with VAD chunks
    state:CardStackState,  # Card stack viewport state
    urls:ReviewUrls,  # URL bundle
) -> List[Any]:  # OOB slot elements
    """Build OOB slot updates for the viewport sections."""
    return build_slots_response(
        card_items=assembled,
        state=state,
        config=REVIEW_CS_CONFIG,
        ids=REVIEW_CS_IDS,
        urls=urls.card_stack,
        render_card=create_review_card_renderer(),
    )

def _build_nav_response(
    assembled:List[AssembledSegment],  # Assembled segments with VAD chunks
    state:CardStackState,  # Card stack viewport state
    urls:ReviewUrls,  # URL bundle
) -> Tuple:  # OOB elements (slots + progress + focus)
    """Build OOB response for navigation."""
    return build_nav_response(
        card_items=assembled,
        state=state,
        config=REVIEW_CS_CONFIG,
        ids=REVIEW_CS_IDS,
        urls=urls.card_stack,
        render_card=create_review_card_renderer(),
        progress_label="Segment",
        form_input_name="segment_index",
    )

## Navigation Handler

In [ ]:
#| export
def _handle_review_navigate(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    sess,  # FastHTML session object
    direction:str,  # Navigation direction: "up", "down", "first", "last", "page_up", "page_down"
    urls:ReviewUrls,  # URL bundle for review routes
):  # OOB slot updates with progress and focus
    """Navigate to a different segment in the viewport using OOB slot swaps."""
    session_id = get_session_id(sess)
    ctx = _load_review_context(state_store, workflow_id, session_id)
    assembled = _get_assembled_segments(ctx)
    
    state = _build_card_stack_state(ctx)
    renderer = create_review_card_renderer()
    
    result = card_stack_navigate(
        direction=direction,
        card_items=assembled,
        state=state,
        config=REVIEW_CS_CONFIG,
        ids=REVIEW_CS_IDS,
        urls=urls.card_stack,
        render_card=renderer,
        progress_label="Segment",
        form_input_name="segment_index",
    )
    
    _update_review_state(state_store, workflow_id, session_id, focused_index=state.focused_index)
    return result

## Update Viewport Handler

Handler for updating the viewport when card count changes.

In [ ]:
#| export
async def _handle_review_update_viewport(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    request,  # FastHTML request object
    sess,  # FastHTML session object
    visible_count:int,  # New number of visible cards
    urls:ReviewUrls,  # URL bundle for review routes
):  # Full viewport component (outerHTML swap)
    """Update the viewport with a new card count."""
    session_id = get_session_id(sess)
    ctx = _load_review_context(state_store, workflow_id, session_id)
    assembled = _get_assembled_segments(ctx)
    
    state = _build_card_stack_state(ctx)
    renderer = create_review_card_renderer()
    
    result = card_stack_update_viewport(
        visible_count=visible_count,
        card_items=assembled,
        state=state,
        config=REVIEW_CS_CONFIG,
        ids=REVIEW_CS_IDS,
        urls=urls.card_stack,
        render_card=renderer,
    )
    
    # Read is_auto from form data
    form_data = await request.form()
    is_auto_str = form_data.get("is_auto", "false")
    is_auto_mode = is_auto_str.lower() == "true"
    
    _update_review_state(
        state_store, workflow_id, session_id,
        visible_count=state.visible_count,
        is_auto_mode=is_auto_mode,
    )
    return result

## Save Width Handler

In [ ]:
#| export
def _handle_review_save_width(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    sess,  # FastHTML session object
    card_width:int,  # Card stack width in rem
) -> None:  # No response body (swap=none on client)
    """Save the card stack width to server state."""
    session_id = get_session_id(sess)
    review_state = _get_review_state(state_store, workflow_id, session_id)
    current_width = review_state.get("card_width", DEFAULT_CARD_WIDTH)
    state = CardStackState(card_width=current_width)
    card_stack_save_width(state, card_width, REVIEW_CS_CONFIG)
    _update_review_state(state_store, workflow_id, session_id, card_width=state.card_width)

## Router Initialization

Creates the card stack router with navigation and viewport routes.

In [ ]:
#| export
def init_card_stack_router(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    prefix:str,  # Route prefix (e.g., "/workflow/review/card_stack")
    urls:ReviewUrls,  # URL bundle (populated after routes defined)
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize card stack routes for review."""
    router = APIRouter(prefix=prefix)

    # -------------------------------------------------------------------------
    # Navigation
    # -------------------------------------------------------------------------

    @router
    def nav_up(request, sess):
        """Navigate to previous segment."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="up", urls=urls)

    @router
    def nav_down(request, sess):
        """Navigate to next segment."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="down", urls=urls)

    @router
    def nav_first(request, sess):
        """Navigate to first segment."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="first", urls=urls)

    @router
    def nav_last(request, sess):
        """Navigate to last segment."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="last", urls=urls)

    @router
    def nav_page_up(request, sess):
        """Navigate up by page."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="page_up", urls=urls)

    @router
    def nav_page_down(request, sess):
        """Navigate down by page."""
        return _handle_review_navigate(state_store, workflow_id, sess, direction="page_down", urls=urls)

    # -------------------------------------------------------------------------
    # Viewport and Width
    # -------------------------------------------------------------------------

    @router
    async def update_viewport(request, sess, visible_count: int):
        """Update viewport with new card count."""
        return await _handle_review_update_viewport(
            state_store, workflow_id, request, sess, visible_count, urls=urls,
        )

    @router
    def save_width(request, sess, card_width: int):
        """Save card stack width to server state."""
        return _handle_review_save_width(state_store, workflow_id, sess, card_width)

    # -------------------------------------------------------------------------
    # Route Dict
    # -------------------------------------------------------------------------

    routes = {
        "nav_up": nav_up,
        "nav_down": nav_down,
        "nav_first": nav_first,
        "nav_last": nav_last,
        "nav_page_up": nav_page_up,
        "nav_page_down": nav_page_down,
        "update_viewport": update_viewport,
        "save_width": save_width,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()